#### load data and liabrary


In [176]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder

# data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')
passenger_ids = test_data["PassengerId"]


### drop unnecessay columns

In [ ]:
data = data.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1)
test_data = test_data.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis= 1)

In [ ]:
# data["Age"] = data["Age"].fillna(data["Age"].median())
# test_data["Age"] = test_data["Age"].fillna(test_data["Age"].median())

In [ ]:
# data["Embarked"] = data["Embarked"].fillna(data["Embarked"].mode()[0])
# test_data["Embarked"] = test_data["Embarked"].fillna(test_data["Embarked"].mode()[0])

In [ ]:
# print(test_data.isnull().sum())
# test_data["Fare"] = test_data["Fare"].fillna(test_data["Fare"].median())

In [ ]:
print("Missing values in train:\n", data.isnull().sum())
print("Missing values in test:\n", test_data.isnull().sum())

In [ ]:
le = LabelEncoder()
data['Sex'] = le.fit_transform(data['Sex'])
data['Embarked'] = le.fit_transform(data['Embarked'])

#### Split feature (X) and target (Y)

In [ ]:
X = data.drop('Survived', axis=1).to_numpy()
y = data['Survived'].to_numpy()

In [ ]:
print("Feature shape:", X.shape)
print("Target shape:", y.shape)

### Train-test split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, x_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Train : {X_train.shape} | {y_train.shape}")
print(f"Validation : {x_val.shape} | {y_val.shape}")

#### Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
x_val = scaler.fit_transform(x_val)

#### Logistic Regression Baseline in TensorFlow

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

log_rel_model = Sequential([
    Dense(1, input_dim = X_train.shape[1], activation='sigmoid')
])

log_rel_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history_log = log_rel_model.fit(
    X_train, y_train,
    validation_data = (x_val, y_val),
    epochs = 100,
    batch_size=32,
    verbose=1
)

loss, acc = log_rel_model.evaluate(x_val, y_val, verbose=0)
print(f" (Logistic Regression) = Loss : {loss} || Accurasy : {acc}")

#### Neural Network (MLP)

In [ ]:
nn_model = Sequential([
    Dense(16, input_dim = X_train.shape[1], activation='relu'),
    Dense(8, activation='relu' ),
    Dense(1, activation='sigmoid')
])

nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history_nn = nn_model.fit(
    X_train, y_train,
    validation_data=(x_val, y_val),
    epochs = 100,
    batch_size = 32,
    verbose = 1,
)

loss, acc = nn_model.evaluate(x_val, y_val, verbose=0)
print(f" (Neural Network (MLP) ) = Loss : {loss} || Accurasy : {acc}")

#### compare results

In [ ]:
plt.plot(history_log.history['val_accuracy'], label="LogReg Val Acc")
plt.plot(history_nn.history['val_accuracy'], label="NN Val Acc")
plt.xlabel("Epochs")
plt.ylabel("Validation Accuracy")
plt.legend()
plt.show()

#### Predictions & Submissio

In [173]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [168]:
X_test_final = scaler.fit_transform(test_data)

In [170]:
y_pred = nn_model.predict(X_test_final)
y_pred_final = (y_pred > 0.5).astype(int).flatten()

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [177]:
submission = pd.DataFrame({
    'passengerID' : passenger_ids,
     "Survived": y_pred_final
})

submission.to_csv("submission.csv", index=False)

print("✅ Submission file created: submission.csv")

✅ Submission file created: submission.csv
